stacking


xtrain, ytrain ->model -> xtest -> pred -> pred,ytest 비교

data ->sum       ->예측값
     -> RF       ->예측값     =>training data로 사용
     ->light GBM ->예측값
     
     
     ->일반적인 vanila stacking 기법
     
     
     
일반적으로 CV 기반 stacking이 많이 사용됨

하나의 특징에 대해서 3개의 fold가 나온다.


In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [224]:
data = pd.read_csv('titanic/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [225]:
data['Embarked'].fillna('S', inplace = True)
data['Fare'].fillna(0, inplace=True)
data['Fare'] = data['Fare'].map(lambda x : np.log(x) if x > 0 else 0)

In [226]:
data['Initial'] = data['Name'].str.extract('([A-Za-z]+)\.')
data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Other'],inplace=True)
mapping = {
    "Mr":0,
    "Miss":1,
    "Mrs" : 1,
    "Master":2,
    "Other":3
}

data['Initial'] = data['Initial'].map(mapping)

In [181]:
mapping_sex = {
    'male' : 0,
    'female': 1
}

mapping_em = {
    'S' :0,
    'C' :1,
    'Q' :2
}


data['Sex'] = data['Sex'].map(mapping_sex)
data['Embarked'] = data['Embarked'].map(mapping_em)


data.drop(['PassengerId', "Ticket", "Cabin", "Name"], axis = 1, inplace = True)

In [182]:
data.groupby('Initial')['Age'].mean()

Initial
0    32.739609
1    27.834615
2     4.574167
3    45.888889
Name: Age, dtype: float64

In [183]:
data.loc[ (data['Age'].isnull()) & (data['Initial'] == 0), 'Age' ] = 32
data.loc[ (data['Age'].isnull()) & (data['Initial'] == 1), 'Age' ] = 28
data.loc[ (data['Age'].isnull()) & (data['Initial'] == 2), 'Age' ] = 5
data.loc[ (data['Age'].isnull()) & (data['Initial'] == 3), 'Age' ] = 45

#나이 반올림

In [184]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Initial
0,0,3,0,22.0,1,0,1.981001,0,0
1,1,1,1,38.0,1,0,4.266662,1,1
2,1,3,1,26.0,0,0,2.070022,0,1
3,1,1,1,35.0,1,0,3.972177,0,1
4,0,3,0,35.0,0,0,2.085672,0,0


In [185]:
y = data['Survived']
X = data.drop('Survived', axis = 1)

In [186]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [187]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
#X_train, y_train로 모델을 만들었다

RandomForestClassifier(random_state=0)

In [188]:
pred = rf.predict(X_test)
print("정확도 :{0:.3f}".format(accuracy_score(y_test, pred)))

정확도 :0.832


In [189]:
#그리드서치 적용해보기. 적용하려면 parameter설정해야함

gb_param_grid = {
    'n_estimators' : [100, 200],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [3, 5, 7, 10],
    'min_samples_split' : [2, 3, 5, 10]
}

In [190]:
gb = GradientBoostingClassifier(random_state=0)
gb.fit(X_train, y_train)

GradientBoostingClassifier(random_state=0)

In [191]:
gb_grid = GridSearchCV(gb, param_grid = gb_param_grid, scoring="accuracy", n_jobs= -1, verbose = 1)
gb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


GridSearchCV(estimator=GradientBoostingClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [3, 5, 7, 10],
                         'min_samples_split': [2, 3, 5, 10],
                         'n_estimators': [100, 200]},
             scoring='accuracy', verbose=1)

In [192]:
gb_grid.best_score_

0.8272234807446074

In [193]:
gb_grid.best_params_
#정확도가 가장 높을 때의 파라미터

{'max_depth': 6,
 'min_samples_leaf': 10,
 'min_samples_split': 2,
 'n_estimators': 100}

In [109]:
#가장 좋은 파라미터들로 모델 생성
#타이타닉의 테스트 데이터를 입력
#결과 제출


In [212]:
test=pd.read_csv('test (1).csv')
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [213]:
test['Embarked'].fillna('S', inplace = True)
test['Fare'].fillna(0, inplace=True)
test['Fare'] = test['Fare'].map(lambda x : np.log(x) if x > 0 else 0)

In [214]:
test['Initial'] = test['Name'].str.extract('([A-Za-z]+)\.')
test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Other'],inplace=True)
mapping = {
    "Mr":0,
    "Miss":1,
    "Mrs" : 1,
    "Master":2,
    "Other":3
}

test['Initial'] = test['Initial'].map(mapping)

In [215]:
mapping_sex = {
    'male' : 0,
    'female': 1
}

mapping_em = {
    'S' :0,
    'C' :1,
    'Q' :2
}


test['Sex'] = test['Sex'].map(mapping_sex)
test['Embarked'] = test['Embarked'].map(mapping_em)


test.drop(['PassengerId', "Ticket", "Cabin", "Name"], axis = 1, inplace = True)

In [216]:
test.groupby('Initial')['Age'].mean()

Initial
0    32.114130
1    30.203095
2     7.406471
3    42.000000
Name: Age, dtype: float64

In [227]:
test.loc[ (test['Age'].isnull()) & (test['Initial'] == 0), 'Age' ] = 32
test.loc[ (test['Age'].isnull()) & (test['Initial'] == 1), 'Age' ] = 28
test.loc[ (test['Age'].isnull()) & (test['Initial'] == 2), 'Age' ] = 5
test.loc[ (test['Age'].isnull()) & (test['Initial'] == 3), 'Age' ] = 45


In [228]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non-null    int64  
 7   Initial   418 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 26.3 KB


In [229]:
test.Age.isnull().sum()

0

In [230]:
test[test.Age.isnull()]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Initial


In [231]:
rf.predict(test)

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [232]:
gb_param_grid = {
    'n_estimators' : [100, 200],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [3, 5, 7, 10],
    'min_samples_split' : [2, 3, 5, 10]
}

In [233]:
gb = GradientBoostingClassifier(random_state=0)
gb.fit(test)

TypeError: BaseGradientBoosting.fit() missing 1 required positional argument: 'y'

https://ictps.or.kr/

https://ictps.or.kr/%EA%B3%B5%EC%A7%80%EC%82%AC%ED%95%AD/12010624

In [135]:
'''
데이터 불균형:클래스가 어느 한 쪽으로만 일방적으로 존재

해결방법

1) 오버샘플링: 클래스가 적은 쪽의 데이터를 랜덤 샘플링하여 복사 붙여넣기를 반복하여 
두 클래스의 비율 비슷하게 함.

2) 언더샘플링: 클래스가 많은 쪽의 데이터를 랜덤 샘플링하여 삭제하기를 반복하여 
두 클래스의 비율 비슷하게 함.

(언더샘플링보다는 오버샘플링을 주로 한다. 언더샘플링은 반영돼야할 데이터가 삭제될 위험이 있기 때문)

3) 오버& 언더 샘플링

ex Y:1000건 vs N:10건 => 1010/2=505, Y는 505건이 될 때까지 언더샘플링, N은 505건이 될 때까지 오버샘플링

4)SMOTE 알고리즘

오버샘플링과 비슷. 
기존 데이터를 적절하게 혼합하여 새로운 데이터를 생성하는 방법

ex)랜덤하게 데이터 선택-> KNN으로 주변의 가까운 주변의 데이터들로부터 새로운 데이터 생성->과정 반복


'''

'\n데이터 불균형:클래스가 어느 한 쪽으로만 일방적으로 존재\n\n해결방법\n\n1) 오버샘플링: 클래스가 적은 쪽의 데이터를 랜덤 샘플링하여 복사 붙여넣기를 반복하여 \n두 클래스의 비율 비슷하게 함.\n\n2) 언더샘플링: 클래스가 많은 쪽의 데이터를 랜덤 샘플링하여 삭제하기를 반복하여 \n두 클래스의 비율 비슷하게 함.\n\n(언더샘플링보다는 오버샘플링을 주로 한다. 언더샘플링은 반영돼야할 데이터가 삭제될 위험이 있기 때문)\n\n3) 오버& 언더 샘플링\n\nex Y:1000건 vs N:10건 => 1010/2=505, Y는 505건이 될 때까지 언더샘플링, N은 505건이 될 때까지 오버샘플링\n\n4)SMOTE 알고리즘\n\n오버샘플링과 비슷. \n기존 데이터를 적절하게 혼합하여 새로운 데이터를 생성하는 방법\n\nex)랜덤하게 데이터 선택-> KNN으로 주변의 가까운 주변의 데이터들로부터 새로운 데이터 생성->과정 반복\n\n\n'

ROC curve->모델의 정확도를 나타낼 수 있다
false positive rate->positive로 예측했는데 틀린 것(0인 케이스를->1로 잘못 예측)
암으로 예측했는데 실제로는 정상
잘못 예측한 값이 0에 가까울수록 종은 모델

true positive rate->1로 예측->1

AUC(area under curve)선 아래 영역 ->auc값이 큰 것이 좋은 모델

kaggle-> credit card fraud detection (신용카드 사기 거래 감지)
결제가 이루어지 전 사기 거래인지 판단하고 거래 중지시킴

roc curve에서 중요한 2가지 요소
민감도SE(sensitivity)=TP(맞춘 양성)/TP+FN(실제 양성) ->FN이 작을수록 정확성이 좋다. 민감도는 클수록 좋다.
특이도SP(specificity)=TN(맞춘 음성)/TN+FP(실제 음성) -> FP가 작을수록 좋다. 특이도가 클수록 좋은 모델

ROC커브:민감도(SE)와 1-특이도(SP)로 그린 그림
x축은 1-특이도,y축은 민감도

1-특이도(SP)=1-TN/TN+FP=TN+FP/TN+FP-TN/TN+FP=FP(양성으로 잘못 예측)/TN+FP(실제 음성)
실제 음성인 것과 양성으로 잘못 예측한 것의 비율=위양성율(FPR)->roc curve에서 x축에 오는 것

DEP-> 검사결과
DS->  검사결과 18점 기준으로 0과 1로  분리


In [235]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [236]:
card_df = pd.read_csv('creditcard.csv')
card_df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


class->0과 1로 이루어짐
train데이터와 test데이터 나뉘어져 있지 않아서 직접 나누어야함

In [237]:
from sklearn.model_selection import train_test_split

트레인과 테스트 사이즈 정해야함.random_state도 정해줘야 매번 같게 나옴

In [238]:
train_test_split(card_df.iloc[:,:-1])

[            Time        V1        V2        V3        V4        V5        V6  \
 191343  129216.0  2.296579 -1.586869 -2.351789 -2.604366  1.175885  3.429416   
 53654    46088.0  1.156134  0.041969  0.161421  1.371922 -0.212523 -0.376359   
 272539  165141.0  2.178798 -1.596805 -2.116136 -1.793119 -0.354999 -0.179154   
 127439   78331.0 -0.577640  0.210301  1.745862 -2.034340 -0.046957  0.072487   
 34013    37471.0  1.125151  0.155549  0.258557  0.485727 -0.003494  0.177396   
 ...          ...       ...       ...       ...       ...       ...       ...   
 234405  147962.0 -4.251648 -2.801520 -0.713997  0.403558  1.977649 -2.065485   
 194950  130837.0  2.233925 -1.534145 -0.524415 -1.525543 -1.714736 -0.683985   
 265377  161860.0 -0.745793  1.010385  1.167394 -1.040777  0.751397 -0.392727   
 242015  151292.0 -0.339345  0.547517  0.455477 -0.236321  0.108504 -0.721153   
 277273  167563.0  2.038994 -0.042528 -3.056462  0.246329  2.945449  3.297981   
 
               V7        V

In [239]:
card_df.iloc[:,:-1]
#class제거

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00


In [240]:
card_df.iloc[:,-1]
#class만 추출

0         0
1         0
2         0
3         0
4         0
         ..
284802    0
284803    0
284804    0
284805    0
284806    0
Name: Class, Length: 284807, dtype: int64

In [241]:
x_features=card_df.iloc[:,:-1] #x, 284807 rows × 30 columns
y_target=card_df.iloc[:,-1] #y, Length: 284807

In [242]:
xtrain,xtest,ytrain,ytest=train_test_split(x_features,y_target,test_size=0.3,random_state=20231023)
#train ->x,y test->x,y 로 나뉨. 그래서 저장하는 변수 꼭 4개로 설정해야함->xtrain,xtest,ytrain,ytest

In [243]:
ytrain

218489    0
56731     0
227772    0
86066     0
17        0
         ..
65529     0
125641    0
152463    0
148223    0
203495    0
Name: Class, Length: 199364, dtype: int64

In [244]:
ytrain.value_counts()

0    199028
1       336
Name: Class, dtype: int64

In [245]:
ytest.value_counts()

0    85287
1      156
Name: Class, dtype: int64

In [246]:
xtrain,xtest,ytrain,ytest=train_test_split(x_features,y_target,test_size=0.3,
                                           random_state=20231023,stratify=y_target)

In [247]:
ytrain.value_counts()

0    199020
1       344
Name: Class, dtype: int64

In [248]:
ytest.value_counts()

0    85295
1      148
Name: Class, dtype: int64

In [249]:
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    df_copy.drop('Time', axis=1, inplace=True)
    return df_copy

In [250]:
df_copy=get_preprocessed_df(card_df)

In [251]:
df_copy

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [252]:
def get_train_test_dataset(df=None):
    df_copy = get_preprocessed_df(df)
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.3, random_state=0, stratify=y_target)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)

In [255]:
X_train
X_test.shape

(85443, 29)

In [256]:
print(y_train.value_counts()/y_train.shape[0] * 100)

0    99.827451
1     0.172549
Name: Class, dtype: float64


In [257]:
print(y_test.value_counts()/y_test.shape[0] * 100)

0    99.826785
1     0.173215
Name: Class, dtype: float64


In [258]:
from sklearn.linear_model import LogisticRegression

In [259]:
lr_clf=LogisticRegression()

In [260]:
lr_clf.fit(X_train,y_train)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [261]:
lr_clf.predict(X_test)
#LogisticRegression()->이진분류를 했다.

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

딥러닝 기초 63p
선형회귀
regression ->선형(직선 한 개)
좋은 직선->직선과 점이 떨어진 거리가 작은 직선


cost function

H(x)->예측값
y->실제값

Gradient Descent Algorithm


Logistic classification(regression)
127p
0또는 1로 분류작업



142p
Softmax Cross-Entropy

Multinomial Classification(다중분류)

클래스의 종류만큼 분류기 만든다
가까이 있는 것으로 분류됨

Softmax function
각각의 분류기에서 연산한 결과값을 전달받아 확률로 변환해주는 함수

linear regression->logits classification->softmax->다중분류가 가능해짐



In [262]:
pd.Series(lr_clf.predict(X_test))

0        0
1        0
2        0
3        0
4        0
        ..
85438    0
85439    0
85440    0
85441    0
85442    0
Length: 85443, dtype: int64

In [263]:
pd.Series(lr_clf.predict(X_test)).value_counts()

0    85341
1      102
dtype: int64

In [264]:
lr_clf.predict_proba(X_test)
#한 행의 합이 1 ->확률로 나온 것


array([[9.98651690e-01, 1.34831002e-03],
       [9.99876546e-01, 1.23454097e-04],
       [9.99808217e-01, 1.91783045e-04],
       ...,
       [9.99753313e-01, 2.46686772e-04],
       [9.99253336e-01, 7.46664288e-04],
       [9.99865249e-01, 1.34751489e-04]])

In [265]:
lr_clf.predict_proba(X_test)[:,1]

array([0.00134831, 0.00012345, 0.00019178, ..., 0.00024669, 0.00074666,
       0.00013475])

In [269]:
#lightgbm->xgboost보다 대체로 성능 좋음

In [268]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [273]:
from lightgbm import LGBMClassifier

In [274]:
lgbm_clf=LGBMClassifier(n_estimators=1000,num_leaves=64,boost_from_average=False)
#boost_from_average=는 데이터가 불균형 분포되어 있을 때 false.
#데이터가 균형 갖도록 내부적으로 보정하는 옵션

In [275]:
lgbm_clf.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 344, number of negative: 199020
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008390 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 199364, number of used features: 29
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

LGBMClassifier(boost_from_average=False, n_estimators=1000, num_leaves=64)

In [277]:
pred=lgbm_clf.predict(X_test)
pred
#test데이터 예측결과

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [279]:
lgbm_clf.predict_proba(X_test)
#확률

array([[9.99999997e-01, 3.49830240e-09],
       [1.00000000e+00, 1.08517297e-10],
       [1.00000000e+00, 2.23232317e-10],
       ...,
       [9.99999996e-01, 3.66393546e-09],
       [9.99999999e-01, 6.63996306e-10],
       [1.00000000e+00, 1.08138237e-10]])

In [ ]:
#데이터 이상치 제거
#데이터 표준화

In [281]:
X_train.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
211605,-8.367621,7.402969,-5.114191,-2.966792,-0.985904,-1.660018,0.397816,1.008250,5.290976,9.315735,...,3.589299,-0.557927,0.349087,0.301734,0.662330,1.145939,-0.012273,1.513736,0.669504,0.69
231628,-1.290662,0.764452,-0.169233,-1.688528,1.984718,-0.717210,1.667635,-0.228295,-0.959071,-0.679429,...,0.057620,0.138615,0.390374,-0.678774,0.292254,1.425865,0.265341,0.051405,0.103879,55.47
110643,-0.606860,0.233923,2.633361,-0.155762,-0.328850,-0.070593,0.173023,0.113394,0.680147,-0.686651,...,-0.129287,0.007393,0.265045,-0.040615,0.406088,-0.498823,0.121851,-0.046840,-0.087259,11.50
11478,1.374168,-0.330389,0.812971,-0.183112,-1.074771,-0.933788,-0.544238,-0.348237,0.614925,0.080408,...,0.078163,-0.134742,0.016793,0.002610,0.714527,0.536175,-0.307487,-0.003555,0.013024,10.00
243724,-2.146505,-7.603113,-4.928224,1.449173,-2.083871,-0.984759,3.629816,-1.302973,-1.434148,-0.021909,...,4.009181,1.274986,-0.857670,-2.109682,0.169338,-0.322888,0.777822,-0.552258,0.304777,2290.05


In [282]:
X_train.describe()
#평균과 중위수가 거의 같다.

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,...,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000
mean,0.000386,-0.000637,0.000587,-0.001239,0.000596,0.000806,-0.000127,0.001900,0.000387,-0.000203,...,0.000546,0.001181,0.000770,-0.000907,0.000708,0.000160,-0.001173,-0.000476,0.000252,88.286313
std,1.959976,1.658734,1.512912,1.413217,1.362707,1.322962,1.216585,1.191216,1.098032,1.086637,...,0.774205,0.736703,0.724805,0.630090,0.605543,0.520830,0.482453,0.401370,0.330830,248.033917
min,-56.407510,-72.715728,-32.965346,-5.683171,-35.182120,-26.160506,-43.557242,-73.216718,-13.434066,-24.588262,...,-28.009635,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-11.710896,0.000000
25%,-0.921490,-0.598848,-0.889918,-0.849806,-0.690631,-0.767513,-0.556268,-0.207917,-0.642713,-0.535148,...,-0.211459,-0.227817,-0.540206,-0.162056,-0.353481,-0.317180,-0.328089,-0.070961,-0.053200,5.640000
50%,0.015893,0.065727,0.180635,-0.019842,-0.054758,-0.273142,0.040109,0.023008,-0.050546,-0.091521,...,-0.062514,-0.028990,0.007585,-0.011301,0.041293,0.016244,-0.053516,0.001207,0.011043,22.000000
75%,1.316633,0.804333,1.028314,0.744065,0.609794,0.400874,0.570132,0.327720,0.597585,0.456096,...,0.133886,0.186863,0.529286,0.147035,0.440205,0.351141,0.240123,0.090888,0.078216,77.000000
max,2.454930,22.057729,9.382558,16.875344,34.801666,21.550496,36.877368,20.007208,15.594995,23.745136,...,39.420904,27.202839,10.503090,22.528412,4.022866,6.070850,3.463246,12.152401,33.847808,19656.530000


In [283]:
import numpy as np

In [290]:
#이상치 제거
q25=np.percentile(X_train['V1'].values, 25)
q75=np.percentile(X_train['V1'].values, 75)
iqr=q75-q25
iqr15=iqr*1.5
#표준화

#np.percentile->백분위 특정하면 그 위치의 값 알려주는 함수


In [291]:
lowest_val=q25-iqr15 #하한 바운더리
highest_val=q75+iqr15#상한 바운더리

In [287]:
X_train['V1']<lowest_val

211605     True
231628    False
110643    False
11478     False
243724     True
          ...  
88290     False
22346     False
269968    False
175960    False
47973     False
Name: V1, Length: 199364, dtype: bool

In [288]:
X_train['V1'][X_train['V1']<lowest_val]

211605    -8.367621
243724    -2.146505
151631    -7.824182
140340    -2.303570
134412    -3.196156
            ...    
20260    -10.648382
19760    -14.191832
238393    -2.919119
160211    -3.087943
52614     -2.741370
Name: V1, Length: 17664, dtype: float64

In [289]:
X_train['V1'][X_train['V1']>highest_val]

155329    2.439207
193547    2.454930
230533    2.451888
Name: V1, dtype: float64

In [292]:
X_train['V1'][(X_train['V1']<lowest_val) | (X_train['V1']>highest_val)]

211605    -8.367621
151631    -7.824182
76190     -6.078681
207772    -7.794335
220170    -5.121007
            ...    
125442    -8.836906
264145    -4.692089
216442   -16.536406
20260    -10.648382
19760    -14.191832
Name: V1, Length: 4928, dtype: float64

In [293]:
X_train['V1'][(X_train['V1']<lowest_val) | (X_train['V1']>highest_val)].index

Int64Index([211605, 151631,  76190, 207772, 220170, 109519, 166031, 283347,
             18527, 259701,
            ...
            222507,  72745,  20946,  77564, 169338, 125442, 264145, 216442,
             20260,  19760],
           dtype='int64', length=4928)

In [295]:
#나중에 타이타닉도 적용해보기

In [296]:
#SMOTE 오버샘플링


In [297]:
pip install -U imbalanced-learn

  Obtaining dependency information for imbalanced-learn from https://files.pythonhosted.org/packages/a3/9e/fbe60a768502af54563dcb59ca7856f5a8833b3ad5ada658922e1ab09b7f/imbalanced_learn-0.11.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/235.6 kB ? eta -:--:--
   ---------------------------------------- 235.6/235.6 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1
Note: you may need to restart the kernel to use updated packages.


In [298]:
from imblearn.over_sampling import SMOTE

In [299]:
smote=SMOTE(random_state=0)
#동일 데이터들 선택됨

In [300]:
X_train_over,y_train_over=smote.fit_resample(X_train,y_train)

In [301]:
X_train.shape

(199364, 29)

In [303]:
X_train_over.shape

(398040, 29)

In [304]:
pd.Series(y_train_over).value_counts()

0    199020
1    199020
Name: Class, dtype: int64

In [305]:
lr_clf=LogisticRegression()

In [306]:
lr_clf.fit(X_train_over,y_train_over)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [307]:
lr_clf.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [308]:
#stacking

In [309]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [310]:
cancer_data=load_breast_cancer()
cancer_data

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [318]:
X_data=cancer_data.data
X_data
#독립변수

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [316]:
y_label=cancer_data.target
y_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [319]:
np.shape(X_data)

(569, 30)

In [320]:
X_train, X_test, y_train,y_test=train_test_split(X_data, y_label, test_size=0.2, random_state=20231024)

In [326]:
#stacking을 하려면 기본 모델을 만들어야함
#기본모델(knn,rf,dt,adaboost)
knn_clf=KNeighborsClassifier(n_neighbors=5)
rf_clf=RandomForestClassifier(n_estimators=100, random_state=42)
dt_clf=DecisionTreeClassifier()
ada_clf=AdaBoostClassifier(n_estimators=100)

In [327]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [328]:
knn_pred=knn_clf.predict(X_test)
rf_pred=rf_clf.predict(X_test)
dt_pred=dt_clf.predict(X_test)
ada_pred=ada_clf.predict(X_test)

In [333]:
print(accuracy_score(y_test,knn_pred))
print(accuracy_score(y_test,rf_pred))
print(accuracy_score(y_test,dt_pred))
print(accuracy_score(y_test,ada_pred))

0.9298245614035088
0.9473684210526315
0.9736842105263158
0.9736842105263158


In [330]:
knn_pred

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1])

In [331]:
rf_pred

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0])

In [332]:
dt_pred

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 1])

In [334]:
ada_pred

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 1])

In [335]:
[knn_pred,rf_pred,dt_pred,ada_pred]

[array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
        1, 1, 1, 1]),
 array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 0, 0]),
 array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 

In [338]:
pred=np.array([knn_pred,rf_pred,dt_pred,ada_pred])
pred
#2차원 구조 array가 된다.

array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
        1, 1, 1, 1],
       [1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,

In [339]:
pred.shape

(4, 114)

In [340]:
pred=np.transpose(pred)
pred

array([[1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1,

In [341]:
pred.shape

(114, 4)

In [342]:
#최종 스태킹 모델:로지스틱 회귀

In [343]:
lr_final=LogisticRegression()

In [344]:
lr_final.fit(pred,y_test)

LogisticRegression()

In [345]:
final=lr_final.predict(pred)
final

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 1])

In [346]:
accuracy_score(final,y_test)

0.9824561403508771

Assosiation Rule pdf

Association Rule Mining ->장바구니 분석이라고도 부름
연관규칙 

Itemset:항목 집합

Support count (θ): frequency of occurrence of an itemset – θ({milk, bread, diaper}) = 2
->4,5번에 항목 집합이 있다.

Support (S): fraction of transactions that contain an 
itemset – S({milk, bread, diaper}) = 2/5 (milk가 여러개 있어도 1로 침)

->전체 거래중에서 항목 집합의 빈도수

Frequent itemset:빈발 항목 집합:최소 지지도 이상 항목집합
예)minimum support threshold를 2라ㅏ고 설정했다면 bread.milk는
묶음으로 3번 등장했으므로 빈발항목집합이라고 할 수 있다.
beer,coke는 1번 등장->빈발항목집합 아님

 Support(신뢰도)
 confidence(지지도)
 
 Frequent Itemset Generation
 minsup ->직접 설정
 
 바탕으로 Rule Generation
 (빈발 아이템셋으로부터 룰 생성)
 
Given d items, there 
are 2d possible 
candidate itemsets

{A,B,C,D,E} 2^5=32
.
.
.
숫자가 너무 커져서 연산 불가


Apriori Principle





지지도를 만족->빈발항목집합
만족X->비빈발


https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/
내용 이해 안가면 참고   

삼겸살과 상추사이의 관계가 궁금하다.

s(지지도)=2/5=0.4
상추,사이다 s=1/5=0.2
(시간 순서 고려 X)

삼겹살,상추 c=2/3

or

상추지지도/삼겹살 지지도 

X->Y

c(X->Y)=0.9

s(Y)=0.9

사실상 X랑Y 가 아무관계없음
왜? Y의 지지도가 높기 때문에
c(X->Y)=0.9라고 해서 X와 Y의 연관성이 있는 것이 아님
기존에 Y를 구매하는 사람이 많아서 높게 나온 것

신뢰도와 지지도만으로는 연관규칙을 찾을 수 없다.
그래서 lift(향상도) 사용해야함

향상도(삼겹살->상추)=  c(삼겹살->상추) / s(상추)=(2/3) / (2/5)=1.66....


향상도=1 이라면  둘이 관계없다는 의미다 왜냐하면

상추를 사는 사람과 삼겹살을 살 때 상추를 같이 구매하는 사람의 비율이 같은 것이기 때문

향상도<1 이라면 둘이 음의 상관관계를 가지고 있다는 것



leverage(레버리지):상관관계를 차이를 이용해서 나타낼 수 있음

lift(삼겹살->상추)=c(삼겹살->상추)/s(상추)

조건부확률 수식으로 나타내기도 한다.

=p(상추|삼겹살)/p(상추)




leverage(레버리지)=p(삼겹살 교집합 상추)-p(삼겹살)p(상추)

=2/5 -3/5*2/5 =4/25    


레버리지가 0에 가깝다면 두 상품은 관계가 없는 것
레버리지가 0에 가깝다면 두 상품은 독립이다.

레버리지가 0보다 크다면 향상도가 1보다 큰 경우이기 때문에 두 상품은 연관성이 있다.


conviction

convict(삼겹살->상추)=p(상추 여집합) / p(상추 여집합 |삼겹살)
삼겹살을 구매한 사람들 중 상추를 구매하지 않은 사람들 중에 
상추를 구매하지 않은 비율

conviction이 1이면 서로 관련없다.
1보다 크다면 ?
1보다 작으면 삼겹살은 상추 구매여부 판단하는 데에 유용하지 않다


순차분석

data->빈발항목추출->빈발 시퀀스                      

In [348]:
#연관규칙

In [1]:
dataset=[['사과','치즈','생수'],
['생수','호두','치즈','고등어'],
['수박','사과','생수'],
['생수','호두','치즈','옥수수']]

In [2]:
pip install mlxtend

  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/73/da/d5d77a9a7a135c948dbf8d3b873655b105a152d69e590150c83d23c3d070/mlxtend-0.23.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.4 MB 5.3 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.4 MB 4.6 MB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.4 MB 6.5 MB/s eta 0:00:01
   ------------------------------------ --- 1.3/1.4 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------  1.4/1.4 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
te=TransactionEncoder()

In [5]:
te_ary=te.fit(dataset).transform(dataset)

In [6]:
dataset

[['사과', '치즈', '생수'],
 ['생수', '호두', '치즈', '고등어'],
 ['수박', '사과', '생수'],
 ['생수', '호두', '치즈', '옥수수']]

In [7]:
te_ary

array([[False,  True,  True, False, False,  True, False],
       [ True, False,  True, False, False,  True,  True],
       [False,  True,  True,  True, False, False, False],
       [False, False,  True, False,  True,  True,  True]])

In [8]:
te.columns_
#열 이름

['고등어', '사과', '생수', '수박', '옥수수', '치즈', '호두']

In [9]:
te_ary
df=pd.DataFrame(te_ary, columns=te.columns_)
df

,고등어,사과,생수,수박,옥수수,치즈,호두
0,False,True,True,False,False,True,False
1,True,False,True,False,False,True,True
2,False,True,True,True,False,False,False
3,False,False,True,False,True,True,True


In [10]:
#지지도 설정


In [11]:
apriori(df,min_support=0.5)
#min_support 최소지지도 설정


,support,itemsets
0,0.50,(1)
1,1.00,(2)
2,0.75,(5)
3,0.50,(6)
4,0.50,"(1, 2)"
5,0.75,"(2, 5)"
6,0.50,"(2, 6)"
7,0.50,"(5, 6)"
8,0.50,"(2, 5, 6)"


In [12]:
apriori(df,min_support=0.5,use_colnames=True)
#열 이름으로 출력
#support 가 0.5이상인 항목들만 나옴
#빈발항목집합

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(생수, 사과)"
5,0.75,"(생수, 치즈)"
6,0.50,"(호두, 생수)"
7,0.50,"(호두, 치즈)"
8,0.50,"(호두, 생수, 치즈)"


In [13]:
freq_itemsets=apriori(df,min_support=0.5,use_colnames=True)

https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

In [14]:
freq_itemsets

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(생수, 사과)"
5,0.75,"(생수, 치즈)"
6,0.50,"(호두, 생수)"
7,0.50,"(호두, 치즈)"
8,0.50,"(호두, 생수, 치즈)"


In [15]:
#연관규칙 생성 알고리즘 association_rules
association_rules(freq_itemsets)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(사과),(생수),0.50,1.00,0.50,1.0,1.000000,0.000,inf,0.0
1,(치즈),(생수),0.75,1.00,0.75,1.0,1.000000,0.000,inf,0.0
2,(호두),(생수),0.50,1.00,0.50,1.0,1.000000,0.000,inf,0.0
3,(호두),(치즈),0.50,0.75,0.50,1.0,1.333333,0.125,inf,0.5
4,"(호두, 생수)",(치즈),0.50,0.75,0.50,1.0,1.333333,0.125,inf,0.5
5,"(호두, 치즈)",(생수),0.50,1.00,0.50,1.0,1.000000,0.000,inf,0.0
6,(호두),"(생수, 치즈)",0.50,0.75,0.50,1.0,1.333333,0.125,inf,0.5


In [17]:
association_rules(freq_itemsets,metric='lift')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(생수),(사과),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0,0.0
1,(사과),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf,0.0
2,(생수),(치즈),1.00,0.75,0.75,0.750000,1.000000,0.000,1.0,0.0
3,(치즈),(생수),0.75,1.00,0.75,1.000000,1.000000,0.000,inf,0.0
4,(호두),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf,0.0
5,(생수),(호두),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0,0.0
6,(호두),(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf,0.5
7,(치즈),(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5,1.0
8,"(호두, 생수)",(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf,0.5
9,"(호두, 치즈)",(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf,0.0


In [18]:
res=association_rules(freq_itemsets,metric='lift')
res[res['lift']>1]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6,(호두),(치즈),0.50,0.75,0.5,1.000000,1.333333,0.125,inf,0.5
7,(치즈),(호두),0.75,0.50,0.5,0.666667,1.333333,0.125,1.5,1.0
8,"(호두, 생수)",(치즈),0.50,0.75,0.5,1.000000,1.333333,0.125,inf,0.5
10,"(생수, 치즈)",(호두),0.75,0.50,0.5,0.666667,1.333333,0.125,1.5,1.0
11,(호두),"(생수, 치즈)",0.50,0.75,0.5,1.000000,1.333333,0.125,inf,0.5
13,(치즈),"(호두, 생수)",0.75,0.50,0.5,0.666667,1.333333,0.125,1.5,1.0


In [375]:
#호두가격이 올랐다->어떤 것에 영향 주는지 예상
#맥주->땅콩 lift=10 이라면 이때 맥주 가격이 오르면 땅콩 판매율에 영향 있을 것으로 예상할 수 있다.



In [19]:
dataset2= [['Milk', 'Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Ice cream', 'Eggs']]

In [20]:
'''
마트의 데이터분석가라고 생각하고 위의 데이터 분석해서
번들,유통,대체 등
데이터 분석->까페 제출하기

'''

'\n마트의 데이터분석가라고 생각하고 위의 데이터 분석해서\n번들,유통,대체 등\n데이터 분석->까페 제출하기\n\n'

In [21]:
TE=TransactionEncoder()

In [22]:
TE_array=TE.fit(dataset2).transform(dataset2)
TE_array

array([[False, False,  True, False,  True,  True,  True, False,  True],
       [False, False,  True, False, False,  True,  True, False,  True],
       [ True, False,  True, False,  True, False, False, False, False],
       [False,  True, False, False,  True, False, False,  True,  True],
       [False,  True,  True,  True, False, False,  True, False, False]])

In [23]:
TE.columns_

['Apple',
 'Corn',
 'Eggs',
 'Ice cream',
 'Milk',
 'Nutmeg',
 'Onion',
 'Unicorn',
 'Yogurt']

In [24]:
df2=pd.DataFrame(TE_array, columns=TE.columns_)
df2

,Apple,Corn,Eggs,Ice cream,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,True,False,True,True,True,False,True
1,False,False,True,False,False,True,True,False,True
2,True,False,True,False,True,False,False,False,False
3,False,True,False,False,True,False,False,True,True
4,False,True,True,True,False,False,True,False,False


In [25]:
Freq_itemsets=apriori(df2,min_support=0.4,use_colnames=True)
Freq_itemsets

,support,itemsets
0,0.4,(Corn)
1,0.8,(Eggs)
2,0.6,(Milk)
3,0.4,(Nutmeg)
4,0.6,(Onion)
5,0.6,(Yogurt)
6,0.4,"(Eggs, Milk)"
7,0.4,"(Eggs, Nutmeg)"
8,0.6,"(Eggs, Onion)"
9,0.4,"(Eggs, Yogurt)"


In [26]:
association_rules(Freq_itemsets)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Nutmeg),(Eggs),0.4,0.8,0.4,1.0,1.250000,0.08,inf,0.333333
1,(Onion),(Eggs),0.6,0.8,0.6,1.0,1.250000,0.12,inf,0.500000
2,(Nutmeg),(Onion),0.4,0.6,0.4,1.0,1.666667,0.16,inf,0.666667
3,(Nutmeg),(Yogurt),0.4,0.6,0.4,1.0,1.666667,0.16,inf,0.666667
4,"(Eggs, Nutmeg)",(Onion),0.4,0.6,0.4,1.0,1.666667,0.16,inf,0.666667
5,"(Nutmeg, Onion)",(Eggs),0.4,0.8,0.4,1.0,1.250000,0.08,inf,0.333333
6,(Nutmeg),"(Eggs, Onion)",0.4,0.6,0.4,1.0,1.666667,0.16,inf,0.666667
7,"(Eggs, Yogurt)",(Nutmeg),0.4,0.4,0.4,1.0,2.500000,0.24,inf,1.000000
8,"(Eggs, Nutmeg)",(Yogurt),0.4,0.6,0.4,1.0,1.666667,0.16,inf,0.666667
9,"(Yogurt, Nutmeg)",(Eggs),0.4,0.8,0.4,1.0,1.250000,0.08,inf,0.333333


In [27]:
association_rules(Freq_itemsets,metric='lift')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Eggs),(Milk),0.8,0.6,0.4,0.500000,0.833333,-0.08,0.8,-0.500000
1,(Milk),(Eggs),0.6,0.8,0.4,0.666667,0.833333,-0.08,0.6,-0.333333
2,(Eggs),(Nutmeg),0.8,0.4,0.4,0.500000,1.250000,0.08,1.2,1.000000
3,(Nutmeg),(Eggs),0.4,0.8,0.4,1.000000,1.250000,0.08,inf,0.333333
4,(Eggs),(Onion),0.8,0.6,0.6,0.750000,1.250000,0.12,1.6,1.000000
5,(Onion),(Eggs),0.6,0.8,0.6,1.000000,1.250000,0.12,inf,0.500000
6,(Eggs),(Yogurt),0.8,0.6,0.4,0.500000,0.833333,-0.08,0.8,-0.500000
7,(Yogurt),(Eggs),0.6,0.8,0.4,0.666667,0.833333,-0.08,0.6,-0.333333
8,(Milk),(Yogurt),0.6,0.6,0.4,0.666667,1.111111,0.04,1.2,0.250000
9,(Yogurt),(Milk),0.6,0.6,0.4,0.666667,1.111111,0.04,1.2,0.250000


In [28]:
res2=association_rules(Freq_itemsets,metric='lift')
res2[res2['lift']>2]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
22,"(Eggs, Yogurt)",(Nutmeg),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
27,(Nutmeg),"(Eggs, Yogurt)",0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
34,"(Yogurt, Onion)",(Nutmeg),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
39,(Nutmeg),"(Yogurt, Onion)",0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
40,"(Eggs, Yogurt, Onion)",(Nutmeg),0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
44,"(Eggs, Yogurt)","(Nutmeg, Onion)",0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
46,"(Eggs, Nutmeg)","(Yogurt, Onion)",0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
47,"(Yogurt, Onion)","(Eggs, Nutmeg)",0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
49,"(Nutmeg, Onion)","(Eggs, Yogurt)",0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
53,(Nutmeg),"(Eggs, Yogurt, Onion)",0.4,0.4,0.4,1.0,2.5,0.24,inf,1.0
